In [2]:
%load_ext autoreload
%autoreload 2


from rag_components import *
from llama_index.core.retrievers import VectorIndexRetriever

import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/isaac/miniforge3/envs/LongRAG/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for bigainlco/LooGLE contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigainlco/LooGLE
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


## Summarization Specific Components


In [3]:
def summarize_chunks(chunks, summarizer_llm=mixtral, text_title=None):
    """
    Generate a summary for each text chunk using the summarizer_llm.

    Args:
        chunks (list[str]): A list of text chunks to be summarized.

    Returns:
        list[str]: Summaries of the text chunks.
    """
    summaries = []
    if text_title is None:
        text_title = "chunks"
    for chunk in tqdm(chunks, desc=f'Summarizing "{text_title}"', leave=False):
        if len(summaries) == 0:
            prompt = f"""<s>[INST]Summarize the following text conceptually. \
                The summary should paraphrase the original text, be significantly \
                shorter, retain all propositions, and be able to replace the \
                original text. Here is the text to be summarized:\n\n{chunk}[/INST]"""
        else:
            prompt = f"""<s>[INST]Summarize the following text conceptually in the context of the text that precedes it. \
                The summary should paraphrase the original text, be significantly \
                shorter, retain all propositions, and be able to replace the \
                original text. Here is the the context:\n\n{summaries[-1]}\n\n\
                Summarize the following text:\n\n{chunk}[/INST]"""

        prompt = re.sub(r"\s+", " ", prompt)
        response = summarizer_llm.complete(prompt).text.strip()
        summaries.append(response)
    return summaries


def summarize_index(text_title, overwrite_existing=False, summarizer_llm=Settings.llm):
    """
    Generate a summary for each text chunk in the index using the summarizer_llm.

    Args:
        index (VectorStoreIndex): The index containing text chunks to be summarized.

    Returns:
        VectorStoreIndex: An index containing summaries for each text chunk.
    """
    if not overwrite_existing:
        existing_index = get_index_by_title(f"{text_title}_summaries")
        if existing_index:
            return existing_index
    index = get_index_by_title(text_title)
    text_chunk_ids = get_ids_from_index(index)
    id_nums = [int(id_.split("_")[-1]) for id_ in text_chunk_ids]
    summary_ids = [f"summary_{id_num}" for id_num in id_nums]
    chunks = [get_text_by_id(index, node_id) for node_id in text_chunk_ids]
    summaries = summarize_chunks(chunks, summarizer_llm, text_title=text_title)

    summary_index = create_index_from_chunks_with_ids(
        summaries, summary_ids, f"{text_title}_summaries", overwrite_existing=overwrite_existing
    )
    return summary_index


def answer_reading_comprehension_with_summarization(
    question, context_title, context_text, top_k=2, chunk_size=1024, chunk_overlap=200, qa_llm=gpt4
):
    """
    Answer a question given a context.

    Args:
        question (str): The question to answer
        context_title (str): The title of the context
        context_text (str): The text of the context
        top_k (int): The number of top matching chunks to retrieve

    Returns:
        str: The answer to the question
    """

    # Find the top k most relevant summary chunks
    text_index_title = f"{context_title}_chunksize{chunk_size}"
    summary_index = summarize_index(text_index_title, summarizer_llm=mixtral)
    summary_retriever = VectorIndexRetriever(
        index=summary_index,
        similarity_top_k=top_k,
    )
    top_summary_chunks = summary_retriever.retrieve(question)

    # Identify the ids of the corresponding raw text chunks
    retrieved_summary_ids = [chunk.node.id_ for chunk in top_summary_chunks]
    corresponding_text_ids = [f"text_chunk_{summary_id.split('_')[-1]}" for summary_id in retrieved_summary_ids]

    # Get the correponding raw text chunks
    text_index = create_index_from_text_with_ids(
        context_text, text_index_title, chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    corresonding_chunks = [get_node_by_id(text_index, text_id) for text_id in corresponding_text_ids]

    # Find the top k * 10 most relevant raw text chunks, and try to find the rank of the chunks which correspond to the top summary chunks
    text_retriever = VectorIndexRetriever(
        index=text_index,
        similarity_top_k=top_k * 10,
    )
    retrieved_texts = text_retriever.retrieve(question)
    text_chunk_ranks = []
    for i, chunk in enumerate(corresonding_chunks):
        id_ = chunk.id_
        rank = None
        for j, retrieved_text in enumerate(retrieved_texts):
            if retrieved_text.node.id_ == id_:
                rank = j
                break
        text_chunk_ranks.append(rank)

    corresponding_chunks_text = [chunk.text for chunk in corresonding_chunks]
    corresponding_chunks_text_combined = " ".join(corresponding_chunks_text)

    response = answer_reading_comprehension(question, corresponding_chunks_text_combined, qa_llm=qa_llm)

    top_chunks_info = [
        {
            "summary_score": top_summary_chunks[i].score,
            "summary_rank": i,
            "text_score": similarity_score(question, text_index_title, corresonding_chunks[i].id_),
            "text_rank": text_chunk_ranks[i],
            "summary": top_summary_chunks[i].node.text,
            "text": corresonding_chunks[i].text,
        }
        for i in range(len(corresonding_chunks))
    ]

    additional_info = {
        "top_chunks_info": top_chunks_info,
    }
    return response, additional_info

## Inference


In [4]:
debug_lim = 100
chunk_sizes = [64, 128, 256, 512, 1024, 2048]
topks = [32, 16, 8, 4, 2, 1]
chunk_overlaps = [10, 25, 50, 100, 200, 400]
for idx in range(3, 4):
    print("Generating output for chunk size", chunk_sizes[idx])
    output_file = f"output/summarization_with_corresponding_in_context_mistral_large_csize{chunk_sizes[idx]}.jsonl"
    test_longdep_qa(
        answer_reading_comprehension_with_summarization,
        output_file=output_file,
        debug_lim=debug_lim,
        qa_llm=mistral_large,
        chunk_size=chunk_sizes[idx],
        top_k=topks[idx],
        chunk_overlap=chunk_overlaps[idx],
    )

Generating output for chunk size 512


Answering questions:   0%|          | 0/100 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'vector_store'

In [ ]:
for idx in range(6):
    output_file = f"output/summarization_with_corresponding_in_context_mistral_large_csize{chunk_sizes[idx]}.jsonl"
    rouge_metrics = get_rouge_metrics(output_file)
    print("results for chunk size", chunk_sizes[idx])
    print("Rouge Metrics:", rouge_metrics)

    self_score = llm_self_score(output_file, llm=gpt4)
    print("LLM Self-Score:", self_score)